In [16]:

from keras import backend as K
from keras.models import load_model
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import cv2
import time

import pickle

In [17]:
model_path = "conf2-lstm-model.h5"
model = load_model(model_path)

In [18]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

In [19]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')

In [20]:
from jaad_data import JAAD

In [21]:
imdb = JAAD(data_path = jaad_path)

In [22]:
vid = "video_0071"

In [23]:
anno = imdb._get_annotations(vid)

In [24]:
l = [k for k,v in anno['ped_annotations']['0_71_386b'].items()]

In [25]:
len(anno['ped_annotations']['0_71_386b']['frames'])

297

In [26]:
len(anno['ped_annotations']['0_71_386b']['bbox'])

297

In [27]:
l

['bbox', 'behavior', 'frames', 'old_id', 'occlusion']

In [28]:
anno['width']

1920

In [34]:
bb_cross={}
ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]

for i, p in enumerate(ped_b):
    if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
        bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
        print(p)
#print(bb_cross)

0_71_386b


In [33]:
for k,v in bb_cross.items():
    print(k)

0_71_386b


In [37]:
len(bb_cross[ped_b[1]])

297

In [38]:
lstm_seq =  bb_cross[ped_b[1]]

In [39]:
lstm_seq

[[1248.0, 620.0, 1299.0, 747.0],
 [1253.0, 619.0, 1304.0, 748.0],
 [1258.0, 618.0, 1308.0, 749.0],
 [1263.0, 617.0, 1313.0, 750.0],
 [1268.0, 616.0, 1318.0, 751.0],
 [1274.0, 615.0, 1323.0, 753.0],
 [1279.0, 614.0, 1328.0, 754.0],
 [1284.0, 613.0, 1333.0, 755.0],
 [1289.0, 612.0, 1337.0, 756.0],
 [1294.0, 611.0, 1342.0, 757.0],
 [1299.0, 609.0, 1348.0, 757.0],
 [1303.0, 608.0, 1352.0, 758.0],
 [1308.0, 606.0, 1358.0, 758.0],
 [1313.0, 604.0, 1363.0, 758.0],
 [1317.0, 602.0, 1368.0, 758.0],
 [1322.0, 601.0, 1374.0, 759.0],
 [1327.0, 599.0, 1379.0, 759.0],
 [1331.0, 597.0, 1384.0, 759.0],
 [1336.0, 595.0, 1389.0, 759.0],
 [1341.0, 594.0, 1395.0, 760.0],
 [1345.0, 592.0, 1400.0, 760.0],
 [1350.0, 590.0, 1405.0, 760.0],
 [1355.0, 588.0, 1411.0, 760.0],
 [1359.0, 587.0, 1415.0, 761.0],
 [1364.0, 585.0, 1421.0, 761.0],
 [1368.0, 585.0, 1426.0, 762.0],
 [1373.0, 585.0, 1431.0, 763.0],
 [1377.0, 585.0, 1436.0, 764.0],
 [1382.0, 586.0, 1442.0, 765.0],
 [1386.0, 586.0, 1447.0, 766.0],
 [1391.0, 

In [40]:
frames = len(anno['ped_annotations']['0_71_386b']['frames'])

In [41]:
frames

297

In [45]:
df = pd.DataFrame(lstm_seq)
df = df.astype('float32')

In [46]:
#df.loc[0:0, :] = np.array([0] * 4) 

In [87]:
df[1:2] 

,0,1,2,3
1,1253.0,619.0,1304.0,748.0


In [88]:
for i in range(df.shape[0]):
    xmin_ = int (df[i:i+1][0] / 2)
    ymin_ = int (df[i:i+1][1] / 2)
    xmax_ = int (df[i:i+1][2] / 2)
    ymax_ = int (df[i:i+1][3] / 2)
    print(i, xmin_, ymin_, xmax_, ymax_)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:19: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.6.1

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


0 624 310 649 373
1 626 309 652 374
2 629 309 654 374
3 631 308 656 375
4 634 308 659 375
5 637 307 661 376
6 639 307 664 377
7 642 306 666 377
8 644 306 668 378
9 647 305 671 378
10 649 304 674 378
11 651 304 676 379
12 654 303 679 379
13 656 302 681 379
14 658 301 684 379
15 661 300 687 379
16 663 299 689 379
17 665 298 692 379
18 668 297 694 379
19 670 297 697 380
20 672 296 700 380
21 675 295 702 380
22 677 294 705 380
23 679 293 707 380
24 682 292 710 380
25 684 292 713 381
26 686 292 715 381
27 688 292 718 382
28 691 293 721 382
29 693 293 723 383
30 695 293 726 383
31 697 293 728 384
32 700 293 731 384
33 702 293 734 385
34 704 293 736 385
35 706 293 739 386
36 709 293 742 386
37 711 293 744 387
38 713 293 747 387
39 715 293 749 388
40 718 293 752 388
41 721 292 755 389
42 724 292 758 389
43 727 291 761 389
44 730 290 764 390
45 733 290 767 390
46 736 289 770 391
47 739 289 773 391
48 742 288 776 391
49 745 288 779 392
50 748 287 782 392
51 750 286 785 392
52 753 286 788 393
53 

In [48]:
pickle_in = open("min-max-scaler.pkl","rb")
scaler = pickle.load(pickle_in)
print(type(scaler))
print(scaler.data_max_)
print(scaler.data_min_)

<class 'sklearn.preprocessing.data.MinMaxScaler'>
[1900.  953. 1919. 1079.]
[0. 0. 0. 0.]


In [49]:
global batch

In [50]:
lstm_seq = scaler.transform(lstm_seq)

In [58]:
lstm_seq.shape[0]

297

In [52]:
batch = scaler.transform(df)

In [53]:
batch

array([[6.56842075e-01, 6.50577089e-01, 6.76915037e-01, 6.92307697e-01],
       [6.59473654e-01, 6.49527771e-01, 6.79520561e-01, 6.93234481e-01],
       [6.62105233e-01, 6.48478453e-01, 6.81604980e-01, 6.94161265e-01],
       ...,
       [2.10526306e-03, 5.03672585e-01, 5.36737866e-02, 9.99073222e-01],
       [1.05263153e-03, 5.17313717e-01, 4.63783205e-02, 9.99073222e-01],
       [5.26315765e-04, 5.42497347e-01, 4.01250638e-02, 1.00000001e+00]])

In [56]:
frames

297

In [59]:
n_seq = 15

In [93]:
def play():
    global batch
    img_width, img_height = 960, 540
    start_time_video = time.time()
    clip_path = "/home/kara9147/JAAD/JAAD_clips/" + vid + ".mp4"
    cap = cv2.VideoCapture(clip_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print( length )
    
    # Time to read all frames, predict and put bounding boxes around them, and show them.
    i = 0
    total_time = 0.0
    pred = True

    # Capture frame-by-frame
    ret = True
    while(ret and i < lstm_seq.shape[0] - n_seq ):
        ret, origimg = cap.read()
        batch = lstm_seq[i:i + n_seq]
        i = i + 1
        #print("Processing {} th frame".format(i))
        if (ret != False and i < frames ):
            #print(img.shape)
            img = cv2.resize(origimg, (img_width, img_height))
            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            print(y_hat)

            #batch =  np.delete(batch, 0, axis = 0)
            #batch =  np.append(batch, y_hat, axis = 0)
            print(len(batch))
            
            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # 4: Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            print(inv_yhat)
            
            
            
            np.set_printoptions(precision=2, suppress=True, linewidth=90)
            # print("Predicted boxes:\n")
            # print('   class   conf xmin   ymin   xmax   ymax')

            #print(y_pred_decoded)

            #print(time.time() - start_time)

            if pred:
                xmin = int(inv_yhat[0] / 2)
                ymin = int(inv_yhat[1] / 2)
                xmax = int(inv_yhat[2] / 2)
                ymax = int(inv_yhat[3] / 2)
            else:
                xmin = int(bb_cross[ped_b[0]][i][0] / 2)
                ymin = int(bb_cross[ped_b[0]][i][1] / 2)
                xmax = int(bb_cross[ped_b[0]][i][2] / 2)
                ymax = int(bb_cross[ped_b[0]][i][3] / 2)


            print((xmin, ymin))
            print((xmax, ymax))
            # 1920x1080
            #cv2.rectangle(img, (0,  0 ), (1910, 1070), (255, 0, 0), 2)

            # Draw the predicted boxes in blue
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax ), (255, 0, 0), 1)

            xmin_ = int (df[i:i+1][0] / 2)
            ymin_ = int (df[i:i+1][1] / 2)
            xmax_ = int (df[i:i+1][2] / 2)
            ymax_ = int (df[i:i+1][3] / 2)
            
            # Draw the present boxes in green
            cv2.rectangle(img, (xmin_, ymin_), (xmax_, ymax_ ), (0, 255, 0), 1)

            # Display the resulting frame
            cv2.imshow('frame',img)

            # waitKey: 0, wait indefinitely
            if cv2.waitKey(20) & 0xFF == ord('q'):
                break

    end_time_video = time.time()
    print("No of frames: {}".format(i))
    print("Total Time: {}".format(total_time))
    print("fps: {}".format(i / (total_time)))

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [96]:
play()

330
[[0.7  0.63 0.72 0.69]]
15
[1323.46  602.17 1384.98  746.85]
(661, 301)
(692, 373)
[[0.7  0.63 0.72 0.69]]
15
[1327.98  601.09 1390.86  747.82]
(663, 300)
(695, 373)
[[0.7  0.63 0.73 0.69]]
15
[1333.73  599.61 1397.22  748.07]
(666, 299)
(698, 374)
[[0.7  0.63 0.73 0.69]]
15
[1337.85  597.63 1401.9   747.97]
(668, 298)
(700, 373)
[[0.71 0.63 0.73 0.69]]
15
[1342.79  595.63 1407.2   747.8 ]
(671, 297)
(703, 373)
[[0.71 0.62 0.74 0.69]]
15
[1347.29  594.47 1412.68  748.73]
(673, 297)
(706, 374)
[[0.71 0.62 0.74 0.69]]
15
[1352.01  592.95 1418.31  749.08]
(676, 296)
(709, 374)
[[0.71 0.62 0.74 0.69]]
15
[1357.2   590.96 1423.95  748.92]
(678, 295)
(711, 374)
[[0.72 0.62 0.75 0.69]]
15
[1362.53  588.96 1429.84  748.79]
(681, 294)
(714, 374)
[[0.72 0.62 0.75 0.69]]
15
[1364.86  587.73 1433.    749.62]
(682, 293)
(716, 374)
[[0.72 0.62 0.75 0.7 ]]
15
[1371.3   586.34 1440.21  749.97]
(685, 293)
(720, 374)
[[0.72 0.61 0.75 0.7 ]]
15
[1374.54  586.07 1444.35  751.02]
(687, 293)
(722, 375)


[1626.76  558.16 1750.52  848.32]
(813, 279)
(875, 424)
[[0.86 0.59 0.91 0.79]]
15
[1626.15  558.16 1750.68  850.18]
(813, 279)
(875, 425)
[[0.86 0.59 0.91 0.79]]
15
[1625.29  559.03 1751.23  853.17]
(812, 279)
(875, 426)
[[0.86 0.59 0.91 0.79]]
15
[1625.7   559.46 1752.47  855.33]
(812, 279)
(876, 427)
[[0.86 0.59 0.91 0.8 ]]
15
[1625.07  559.09 1753.31  857.93]
(812, 279)
(876, 428)
[[0.86 0.59 0.91 0.8 ]]
15
[1625.16  559.34 1754.11  860.12]
(812, 279)
(877, 430)
[[0.86 0.59 0.91 0.8 ]]
15
[1625.17  559.5  1754.8   861.93]
(812, 279)
(877, 430)
[[0.85 0.59 0.91 0.8 ]]
15
[1622.12  560.14 1752.89  864.71]
(811, 280)
(876, 432)
[[0.85 0.59 0.91 0.8 ]]
15
[1623.39  560.66 1755.24  866.99]
(811, 280)
(877, 433)
[[0.85 0.59 0.91 0.81]]
15
[1623.18  560.5  1755.7   868.7 ]
(811, 280)
(877, 434)
[[0.85 0.59 0.92 0.81]]
15
[1622.99  560.44 1756.5   870.57]
(811, 280)
(878, 435)
[[0.85 0.59 0.92 0.81]]
15
[1622.37  560.46 1756.67  872.42]
(811, 280)
(878, 436)
[[0.85 0.59 0.92 0.81]]
15
[162

[[0.67 0.58 0.79 0.96]]
15
[1275.01  553.44 1521.89 1033.3 ]
(637, 276)
(760, 516)
[[0.66 0.58 0.79 0.96]]
15
[1261.61  553.26 1509.64 1034.12]
(630, 276)
(754, 517)
[[0.66 0.58 0.78 0.96]]
15
[1246.54  552.31 1495.1  1033.89]
(623, 276)
(747, 516)
[[0.65 0.58 0.77 0.96]]
15
[1229.84  551.86 1479.69 1034.95]
(614, 275)
(739, 517)
[[0.64 0.58 0.76 0.96]]
15
[1214.99  551.09 1466.32 1036.18]
(607, 275)
(733, 518)
[[0.63 0.58 0.76 0.96]]
15
[1198.13  551.08 1450.67 1037.69]
(599, 275)
(725, 518)
[[0.62 0.58 0.75 0.96]]
15
[1184.51  550.68 1437.62 1037.9 ]
(592, 275)
(718, 518)
[[0.61 0.58 0.74 0.96]]
15
[1167.29  549.29 1420.67 1037.76]
(583, 274)
(710, 518)
[[0.61 0.58 0.73 0.96]]
15
[1151.59  549.12 1405.83 1038.94]
(575, 274)
(702, 519)
[[0.6  0.58 0.72 0.96]]
15
[1134.9   548.27 1390.05 1040.06]
(567, 274)
(695, 520)
[[0.59 0.58 0.72 0.97]]
15
[1118.61  548.27 1374.95 1041.67]
(559, 274)
(687, 520)
[[0.58 0.57 0.71 0.97]]
15
[1103.2   547.82 1359.73 1041.81]
(551, 273)
(679, 520)
[[0.